In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential ,load_model
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.metrics import Accuracy
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [ ]:
filename = "/content/drive/Shareddrives/GRAPH_VC/MSVD/sents_train_lc_nopunc.txt"
# load descriptions
doc = load_doc(filename)
print(doc[:300])

vid1	a squirrel is eating a peanut in it s shell
vid1	a chipmunk is eating
vid1	a chipmunk is eating a peanut
vid1	a chipmunk is eating a nut
vid1	a squirrel is eating a nut
vid1	a squirrel is eating a whole peanut
vid1	a squirrel is eating a peanut
vid1	a squirrel l is eating nuts
vid1	a small squi


In [ ]:
# parse descriptions
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 1200 


In [ ]:
list(descriptions.keys())[:5]

['vid1', 'vid2', 'vid3', 'vid4', 'vid5']

In [ ]:
len(descriptions['vid1'])

29

In [ ]:
# clean descriptions
clean_descriptions(descriptions)

NameError: ignored

In [ ]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 9462


In [ ]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, '/content/drive/Shareddrives/GRAPH_VC/descriptions.txt')

In [ ]:
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split()[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
filename = '/content/drive/Shareddrives/GRAPH_VC/MSVD/sents_train_lc_nopunc.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 1200


In [ ]:
print(train)
len(train)

{'vid1040', 'vid705', 'vid110', 'vid379', 'vid486', 'vid587', 'vid209', 'vid836', 'vid372', 'vid394', 'vid38', 'vid835', 'vid545', 'vid732', 'vid662', 'vid263', 'vid57', 'vid76', 'vid429', 'vid146', 'vid41', 'vid658', 'vid873', 'vid893', 'vid944', 'vid874', 'vid1023', 'vid43', 'vid39', 'vid160', 'vid322', 'vid737', 'vid443', 'vid716', 'vid756', 'vid93', 'vid509', 'vid345', 'vid473', 'vid786', 'vid984', 'vid1083', 'vid784', 'vid317', 'vid341', 'vid140', 'vid442', 'vid425', 'vid161', 'vid78', 'vid45', 'vid898', 'vid791', 'vid783', 'vid344', 'vid133', 'vid1084', 'vid1146', 'vid1155', 'vid412', 'vid983', 'vid669', 'vid310', 'vid230', 'vid546', 'vid343', 'vid12', 'vid331', 'vid1058', 'vid1095', 'vid1148', 'vid1066', 'vid315', 'vid583', 'vid56', 'vid284', 'vid911', 'vid678', 'vid101', 'vid468', 'vid115', 'vid929', 'vid611', 'vid305', 'vid325', 'vid37', 'vid283', 'vid701', 'vid837', 'vid955', 'vid973', 'vid350', 'vid1063', 'vid1034', 'vid436', 'vid254', 'vid403', 'vid445', 'vid1107', 'vid685'

1200

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions('/content/drive/Shareddrives/GRAPH_VC/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=1200


In [ ]:
print(train_descriptions)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Load videoid to video mapping
doc=load_doc('/content/drive/Shareddrives/GRAPH_VC/MSVD/youtube_video_to_id_mapping.txt')
vidid_to_vidname={}
vidname_to_vidid={}
lines=doc.split('\n')
for line in lines:
  x=line.split(' ')
  if len(x) ==2:
    vid_name=x[0]
    vid_id=x[1]
    vidid_to_vidname[vid_id]=vid_name
    vidname_to_vidid[vid_name]=vid_id
print(len(vidid_to_vidname))

1970


In [ ]:
#vidid_to_vidname

In [ ]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

48774

In [ ]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 8
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 9464 -> 2264


In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [ ]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

2265

In [ ]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 47


In [ ]:
def get_feat_by_vidname(name):
              vid_feat_file_name='i3d_resnet50_v1_kinetics400_'+name+'.avi_feat.npy'
              vid_feat_complete_path='/content/drive/Shareddrives/GRAPH_VC/repo/i3d_features/'+vid_feat_file_name
              photo = np.load(vid_feat_complete_path)
              photo.reshape((1,2048))
              return photo
# '/content/drive/Shareddrives/GRAPH_VC/i3d_features/'

In [ ]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            try:
            # retrieve the photo feature
              vid_feat_file_name='i3d_resnet50_v1_kinetics400_'+vidid_to_vidname[key]+'.avi_feat.npy'
              vid_feat_complete_path='/content/drive/Shareddrives/GRAPH_VC/repo/i3d_features/'+vid_feat_file_name
              photo = np.load(vid_feat_complete_path)
              photo=np.reshape(photo, photo.shape[1])
              n+=1
              for desc in desc_list:
                  # encode the sequence
                  seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                  # split one sequence into multiple X, y pairs
                  for i in range(1, len(seq)):
                      # split into input and output pair
                      in_seq, out_seq = seq[:i], seq[i]
                      # pad input sequence
                      in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                      # encode output sequence
                      out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                      # store
                      X1.append(photo)
                      X2.append(in_seq)
                      y.append(out_seq)
              # yield the batch data
              if n==num_photos_per_batch:
                  # print([[array(X1), array(X2)], array(y)])
                  yield [array(X1), array(X2)], array(y)
                  X1, X2, y = list(), list(), list()
                  n=0
            except:
              continue


In [ ]:
# Load Glove vectors
embeddings_index = {} # empty dictionary
f = open('/content/drive/Shareddrives/GRAPH_VC/glove.6B.300d.txt', encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embedding_dim = 300

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(2265, 300)

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 47)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 47, 300)      679500      input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
_______________________________________________________________________________________

In [ ]:
model.layers[2]

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
# model.compile(loss='categorical_crossentropy', optimizer='adam')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 10
number_photos_per_bath = 3
steps = len(train_descriptions)//number_photos_per_bath

In [ ]:
for i in range(epochs):
    print(i)
    generator = data_generator(train_descriptions, wordtoix, max_length, number_photos_per_bath)
    print('return from gen')
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    print('model ' + str(i) + " done")
    model.save('/content/drive/Shareddrives/GRAPH_VC/model_weights/model_' + str(i) + '.h5')

0
return from gen
400/400 [==============================] - 62s 155ms/step - loss: 3.1071 - accuracy: 0.4092
model 0 done
1
return from gen
400/400 [==============================] - 61s 152ms/step - loss: 2.8306 - accuracy: 0.4319
model 1 done
2
return from gen
400/400 [==============================] - 61s 152ms/step - loss: 2.6144 - accuracy: 0.4522
model 2 done
3
return from gen
400/400 [==============================] - 61s 152ms/step - loss: 2.4496 - accuracy: 0.4711
model 3 done
4
return from gen
400/400 [==============================] - 61s 153ms/step - loss: 2.3238 - accuracy: 0.4868
model 4 done
5
return from gen
400/400 [==============================] - 61s 153ms/step - loss: 2.2245 - accuracy: 0.5013
model 5 done
6
return from gen
400/400 [==============================] - 61s 153ms/step - loss: 2.1481 - accuracy: 0.5121
model 6 done
7
return from gen
400/400 [==============================] - 61s 153ms/step - loss: 2.0788 - accuracy: 0.5222
model 7 done
8
return from ge

In [ ]:
model=load_model('/content/drive/Shareddrives/GRAPH_VC/model_weights/model_9.h5')

In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [ ]:
from os import listdir
from os.path import isfile, join
mypath='/content/drive/Shareddrives/GRAPH_VC/repo/i3d_features/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['i3d_resnet50_v1_kinetics400_d1zdJO3CqVw_1_35.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_d7eGypGOlOc_13_22.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_d7Gs0uGFLh0_5_13.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_D9I1Z6O_mbw_112_119.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_D9I1Z6O_mbw_141_146.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_d_BWuttLRFM_42_52.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_dabI5gfaFm4_19_25.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_DAmOfcvfKZ0_22_32.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_DB-3-TVjrxU_58_75.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_DBvSfds3FH4_76_80.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_DBvSfds3FH4_84_88.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_dc4UltkRJsw_53_59.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_DcDYRpa34W8_11_25.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_De815YpTBic_41_48.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_DDtX5a-OgLQ_1_15.avi_feat.npy', 'i3d_resnet50_v1_kinetics400_dEn5E-TNezw_13_23.avi_feat.npy', 'i3d_r

In [ ]:
n=0
for a in onlyfiles:
  pp=np.load(join(mypath,a))
  # print(pp.shape)
  pp=pp.reshape((1,2048))
  print("Greedy:",greedySearch(pp))
  print(a)
  n+=1
  if n==10:
    break

Greedy: a baby is eating food
i3d_resnet50_v1_kinetics400_d1zdJO3CqVw_1_35.avi_feat.npy
Greedy: a man is using a board
i3d_resnet50_v1_kinetics400_d7eGypGOlOc_13_22.avi_feat.npy
Greedy: a cartoon is performing a dance
i3d_resnet50_v1_kinetics400_d7Gs0uGFLh0_5_13.avi_feat.npy
Greedy: a man is riding a bicycle
i3d_resnet50_v1_kinetics400_D9I1Z6O_mbw_112_119.avi_feat.npy
Greedy: a man is picking a
i3d_resnet50_v1_kinetics400_D9I1Z6O_mbw_141_146.avi_feat.npy
Greedy: a man is singing and singing a song
i3d_resnet50_v1_kinetics400_d_BWuttLRFM_42_52.avi_feat.npy
Greedy: a person is peeling a potato
i3d_resnet50_v1_kinetics400_dabI5gfaFm4_19_25.avi_feat.npy
Greedy: a man is slicing a potato
i3d_resnet50_v1_kinetics400_DAmOfcvfKZ0_22_32.avi_feat.npy
Greedy: a man is boxing
i3d_resnet50_v1_kinetics400_DB-3-TVjrxU_58_75.avi_feat.npy
Greedy: a woman is peeling a potato
i3d_resnet50_v1_kinetics400_DBvSfds3FH4_76_80.avi_feat.npy


In [ ]:
from nltk.translate.bleu_score import corpus_bleu 
!pip install --upgrade nltk
import nltk
nltk.download('wordnet')
# from nltk.translate import meteor_score

     |████████████████████████████████| 1.4MB 15.7MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434675 sha256=c76e7a7916a7233ee05a53ee05649812b3e2f56295a33adee69bd4f12c221f5a
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
actual=[]
predicted = []
for key, desc_list in train_descriptions.items():
      yhat = greedySearch(get_feat_by_vidname(vidid_to_vidname[key]))
      # store actual and predicted
      references = descriptions[key]
      ref = [d.split() for d in references]
      actual.append(ref)
      predicted.append(yhat)

FileNotFoundError: ignored

In [1]:
print(len(predicted))

NameError: ignored

In [ ]:
  print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
  print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
  print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
  print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.802836
BLEU-2: 0.704642
BLEU-3: 0.655435
BLEU-4: 0.539788


In [ ]:
print(len(actual))
print(len(predicted))

585
585


In [ ]:
!pip install pycocoevalcap

     |████████████████████████████████| 104.3MB 132kB/s 


In [ ]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

In [ ]:
class COCOEvalCap:
    def __init__(self,images,gts,res):
        self.evalImgs = []
        self.eval = {}
        self.imgToEval = {}
        self.params = {'image_id': images}
        self.gts = gts
        self.res = res

    def evaluate(self):
        imgIds = self.params['image_id']
        gts = self.gts
        res = self.res

        # =================================================
        # Set up scorers
        # =================================================
        print ('tokenization...')
        tokenizer = PTBTokenizer()
        gts  = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)

        # =================================================
        # Set up scorers
        # =================================================
        print ('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr")
        ]

        # =================================================
        # Compute scores
        # =================================================
        eval = {}
        for scorer, method in scorers:
            print ('computing %s score...'%(scorer.method()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, imgIds, m)
                    print ("%s: %0.3f"%(m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, imgIds, method)
                print ("%s: %0.3f"%(method, score))
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if not imgId in self.imgToEval:
                self.imgToEval[imgId] = {}
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

def calculate_metrics(rng,datasetGTS,datasetRES):
    imgIds = rng
    gts = {}
    res = {}

    imgToAnnsGTS = {ann['image_id']: [] for ann in datasetGTS['annotations']}
    for ann in datasetGTS['annotations']:
        imgToAnnsGTS[ann['image_id']] += [ann]

    imgToAnnsRES = {ann['image_id']: [] for ann in datasetRES['annotations']}
    for ann in datasetRES['annotations']:
        imgToAnnsRES[ann['image_id']] += [ann]

    for imgId in imgIds:
        gts[imgId] = imgToAnnsGTS[imgId]
        res[imgId] = imgToAnnsRES[imgId]

    evalObj = COCOEvalCap(imgIds,gts,res)
    evalObj.evaluate()
    return evalObj.eval

if __name__ == '__main__':
    rng = range(2)
    datasetGTS = {
        'annotations': [{u'image_id': 0, u'caption': u'the man is running'},
                        {u'image_id': 1, u'caption': u'a woman is chopping cucumbers and carrots'}]
        }
    datasetRES = {
        'annotations': [{u'image_id': 0, u'caption': u'the man is playing a guitar'},
                        {u'image_id': 1, u'caption': u'a woman is cutting cucumbers and carrots'}]
        }
    print (calculate_metrics(rng,datasetGTS,datasetRES))

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 13, 'reflen': 11, 'guess': [13, 11, 9, 7], 'correct': [9, 6, 3, 0]}
ratio: 1.181818181710744
Bleu_1: 0.692
Bleu_2: 0.615
Bleu_3: 0.501
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.434
computing Rouge score...
ROUGE_L: 0.740
computing CIDEr score...
CIDEr: 4.014
{'Bleu_1': 0.692307692254438, 'Bleu_2': 0.6145098677474599, 'Bleu_3': 0.5011627948387378, 'Bleu_4': 6.511928251713921e-05, 'METEOR': 0.43436118998539447, 'ROUGE_L': 0.7397959183673469, 'CIDEr': 4.014287395082455}


In [ ]:
print(actual)
print(predicted)

[[['a', 'squirrel', 'is', 'eating', 'a', 'peanut', 'in', 'it', 's', 'shell'], ['a', 'chipmunk', 'is', 'eating'], ['a', 'chipmunk', 'is', 'eating', 'a', 'peanut'], ['a', 'chipmunk', 'is', 'eating', 'a', 'nut'], ['a', 'squirrel', 'is', 'eating', 'a', 'nut'], ['a', 'squirrel', 'is', 'eating', 'a', 'whole', 'peanut'], ['a', 'squirrel', 'is', 'eating', 'a', 'peanut'], ['a', 'squirrel', 'l', 'is', 'eating', 'nuts'], ['a', 'small', 'squirrel', 'is', 'eating', 'a', 'peanut'], ['a', 'small', 'animal', 'is', 'chewing', 'on', 'a', 'nut'], ['an', 'ardilla', 'is', 'eating'], ['a', 'squirrel', 'is', 'eating', 'a', 'nut'], ['a', 'chipmunk', 'is', 'eating', 'some', 'food'], ['the', 'squirrel', 'is', 'eating'], ['a', 'ferret', 'eats', 'a', 'leaf'], ['a', 'hamster', 'is', 'eating', 'a', 'peanut'], ['a', 'ardilla', 'coreana', 'eating'], ['the', 'rabbit', 'is', 'eating'], ['a', 'chipmunk', 'is', 'eating', 'a', 'peanut'], ['a', 'rabbit', 'is', 'eating'], ['a', 'rabbit', 'is', 'eating'], ['a', 'squirrel', '

In [ ]:
num_predictions = len(predicted)

rng = range(num_predictions)

datasetGTS = {'annotations' : []}
datasetRES = {'annotations' : []}

f = 0

l_a = []
l_p = []


def push_back(l, idx, val):
    if val == None:
        l += [{u'image_id' : idx, u'caption' : ''}]
        return l

    if type(val).__name__ == 'str':
        l += [{u'image_id' : idx, u'caption' : val}]
        return l

    if type(val).__name__ == 'list':

        if len(val) == 0:
            l += [{u'image_id' : idx, u'caption' : ''}]
            return l

        if type(val[0]).__name__ == 'str':
            s = ''
            for it in val:
                s += ' ' + it
            l += [{u'image_id' : idx, u'caption' : s}]
            return l
            
        if type(val[0]).__name__ == 'list':

            for tmp in val:

                s = ''
                for it in tmp:
                    s += ' ' + it
                l += [{u'image_id' : idx, u'caption' : s}]
            
            return l
            

for i in range(num_predictions):
    l_a = push_back(l_a, i, actual[i])
    l_p = push_back(l_p, i, predicted[i])


datasetGTS['annotations'] = l_a
datasetRES['annotations'] = l_p

print(datasetGTS)
print(datasetRES)

{'annotations': [{'image_id': 0, 'caption': ' a squirrel is eating a peanut in it s shell'}, {'image_id': 0, 'caption': ' a chipmunk is eating'}, {'image_id': 0, 'caption': ' a chipmunk is eating a peanut'}, {'image_id': 0, 'caption': ' a chipmunk is eating a nut'}, {'image_id': 0, 'caption': ' a squirrel is eating a nut'}, {'image_id': 0, 'caption': ' a squirrel is eating a whole peanut'}, {'image_id': 0, 'caption': ' a squirrel is eating a peanut'}, {'image_id': 0, 'caption': ' a squirrel l is eating nuts'}, {'image_id': 0, 'caption': ' a small squirrel is eating a peanut'}, {'image_id': 0, 'caption': ' a small animal is chewing on a nut'}, {'image_id': 0, 'caption': ' an ardilla is eating'}, {'image_id': 0, 'caption': ' a squirrel is eating a nut'}, {'image_id': 0, 'caption': ' a chipmunk is eating some food'}, {'image_id': 0, 'caption': ' the squirrel is eating'}, {'image_id': 0, 'caption': ' a ferret eats a leaf'}, {'image_id': 0, 'caption': ' a hamster is eating a peanut'}, {'ima

In [ ]:
evaluation = calculate_metrics(rng,datasetGTS,datasetRES)
print(evaluation)

tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 3738, 'reflen': 3631, 'guess': [3738, 3153, 2568, 1983], 'correct': [3001, 1950, 1265, 689]}
ratio: 1.0294684659870479
Bleu_1: 0.803
Bleu_2: 0.705
Bleu_3: 0.625
Bleu_4: 0.540
computing METEOR score...
METEOR: 0.379
computing Rouge score...
ROUGE_L: 0.755
computing CIDEr score...
CIDEr: 1.049
{'Bleu_1': 0.8028357410377734, 'Bleu_2': 0.7046422334349789, 'Bleu_3': 0.6253804227146927, 'Bleu_4': 0.539923634007188, 'METEOR': 0.3788573321320704, 'ROUGE_L': 0.7546074354578926, 'CIDEr': 1.0493366263750816}


In [ ]:
# evaluation['CIDEr'] /= 10
for metric in evaluation:
    print(metric + "  Score  =  ", evaluation[metric])

Bleu_1  Score  =   0.8028357410377734
Bleu_2  Score  =   0.7046422334349789
Bleu_3  Score  =   0.6253804227146927
Bleu_4  Score  =   0.539923634007188
METEOR  Score  =   0.3788573321320704
ROUGE_L  Score  =   0.7546074354578926
CIDEr  Score  =   0.10493366263750817


# Bleu_1  Score  =   0.8028357410377734
# Bleu_2  Score  =   0.7046422334349789
# Bleu_3  Score  =   0.6253804227146927
# Bleu_4  Score  =   0.539923634007188
# METEOR  Score  =   0.3788573321320704
# ROUGE_L  Score  =   0.7546074354578926
# CIDEr  Score  =   1.0493366263750816

In [ ]:
!pip install keras-gcn

  Created wheel for keras-gcn: filename=keras_gcn-0.14.0-cp36-none-any.whl size=6492 sha256=5d0df436369ddee204b2be2949da2ed7af869df9ffade1e515bdf9a62f0d9c6b
  Stored in directory: /root/.cache/pip/wheels/fc/ed/5a/f865787d7cf02628074d9c9f3e71d865865067bc5c5343c9da
Successfully built keras-gcn
